# Periodic download of IEX stock-ticker data

In [3]:
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()
stop_minutes = 10
stop_at = now + Δ(minutes=stop_minutes)
print('Running at %s (stopping within %dmins)' % (now.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes)

'2019-11-05'

## Read IEX API credentials from `~/.config/iex.ini`

In [1]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [2]:
ticker = 'AAPL'
tickers = [ticker]
len(tickers)

1

In [4]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [5]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [6]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = date(2019, 4, 1)
end_date = today + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 157 days ([2019-04-01,2019-11-06)), 390 minutes per day


In [7]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [8]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [9]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
    for ticker in tickers
); None

CPU times: user 572 ms, sys: 202 ms, total: 773 ms
Wall time: 8.5 s


In [10]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [11]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [12]:
def load_ticker_date_df(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [13]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [14]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 4.6 s, sys: 88.4 ms, total: 4.69 s
Wall time: 4.8 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.60,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.98,190.980,190.64,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.93,191.090,190.78,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.83,191.010,190.76,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.70,190.760,190.60,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-11-05 10:11:00,256.840,256.84,256.840,256.84,256.840,100,25684.000,1
2019-11-05 10:12:00,256.770,256.95,256.980,256.77,256.906,620,159282.060,9
2019-11-05 10:13:00,257.020,257.10,257.100,257.01,257.044,704,180959.080,7


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [15]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [16]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [17]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
...,...,...,...,...,...,...,...,...,...
2019-11-04 13:08:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.310
2019-11-04 13:22:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.510
2019-11-04 13:46:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,256.900


Spot-check: consecutive pairs of minutes where no trades occurred

In [18]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-23 14:16:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,202.66
...,...,...,...,...,...,...,...,...,...
2019-10-28 13:28:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-10-28 13:55:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,248.51
2019-10-28 13:56:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [19]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              56007
close             56007
high              56007
low               56007
average           56007
volume            56803
notional          56803
numberOfTrades    56803
prev_close        56803
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [20]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [21]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
...,...,...,...,...,...,...,...,...,...
2019-11-04 13:46:00,256.900,256.900,256.900,256.900,256.900,0,0.000,0,256.900
2019-11-04 15:12:00,257.130,257.130,257.130,257.070,257.098,605,155544.450,7,257.135
2019-11-04 15:13:00,257.130,257.130,257.130,257.130,257.130,0,0.000,0,257.130


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [23]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
2019-11-04 13:47:00,0.000311,-0.000078,0.000039,-0.000078,0.000027
2019-11-04 15:13:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-11-04 15:14:00,-0.000194,-0.000370,0.000000,-0.000389,-0.000280


In [24]:
lg.count()

open              56803
close             56803
high              56803
low               56803
average           56803
volume            56803
notional          56803
numberOfTrades    56803
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [25]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-11-05 10:11:00,-0.000778,0.000000,0.000000,0.000000,0.000000,1.000000,4.615121
2019-11-05 10:12:00,-0.000273,0.000701,0.000818,0.000000,0.000530,3.000000,4.246907
2019-11-05 10:13:00,0.000272,0.000311,0.000311,-0.000039,0.000093,2.645751,4.620762


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [26]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-11-05 10:11:00,-0.000778,0.000000,0.000000,0.000000,0.000000,1.000000,4.615121,0.000428
2019-11-05 10:12:00,-0.000273,0.000701,0.000818,0.000000,0.000530,3.000000,4.246907,0.000584
2019-11-05 10:13:00,0.000272,0.000311,0.000311,-0.000039,0.000093,2.645751,4.620762,0.000661


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [27]:
lg.dropna(how='any', inplace=True)
lg.count()

open          56656
close         56656
hi            56656
lo            56656
avg           56656
trades        56656
avgVol        56656
next_close    56656
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [88]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
tf, vf = tn / n, vn / n
n = tn + vn
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

45134 training samples (79.66%), 11522 validation (20.34%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [80]:
lg_t = lg[~vfs]
lg_v = lg[ vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,7.644329e-06,0.000232
close,-4.794258e-06,0.000573
hi,2.737419e-04,0.000424
lo,-2.844005e-04,0.000454
avg,-1.034878e-06,0.000359
trades,3.696078e+00,1.768256
avgVol,4.402907e+00,0.751627
next_close,-6.214245e-07,0.000611


Helpers for showing descriptive statistics of each column of a DataFrame:

In [81]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [82]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,56656,6.824323e-06,0.000231,-0.491489,16.296609,-0.006159,0.002457
close,56656,-4.190837e-06,0.000571,-0.172631,47.422334,-0.018992,0.014990
hi,56656,2.742036e-04,0.000426,4.882894,60.358916,0.000000,0.014990
lo,56656,-2.841045e-04,0.000448,-7.031772,183.155173,-0.023854,0.000000
avg,56656,-6.697426e-07,0.000357,-1.116430,77.223916,-0.015078,0.007264
trades,56656,3.697636e+00,1.770064,1.126718,3.159000,0.000000,21.679483
avgVol,56656,4.403211e+00,0.750722,-3.470566,18.580684,0.000000,10.533455
next_close,56656,6.768027e-07,0.000609,-0.394536,38.142905,-0.018822,0.015714


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [83]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.032930,-9.077997,-0.645317,-11.427499,-6.628619,1.661056,0.258170,2.830562
2019-04-01 09:31:00,1.096683,2.569451,2.813446,-0.066756,0.893182,1.108876,0.301141,-0.427388
2019-04-01 09:32:00,1.771186,-1.179740,-0.275191,-2.606325,-1.585695,1.007291,-0.094274,-0.856129
2019-04-01 09:33:00,1.095026,-1.363172,-0.275036,-1.914133,-0.492748,0.848335,0.037132,-1.113945
2019-04-01 09:34:00,-1.613393,-0.540751,-0.645317,-1.222935,-1.369184,-0.131190,0.130155,1.287443
...,...,...,...,...,...,...,...,...
2019-11-05 10:10:00,0.134665,0.008369,-0.645317,0.627010,0.002881,-1.524711,-4.396188,-1.272526
2019-11-05 10:11:00,-3.386070,0.008369,-0.645317,0.627010,0.002881,-1.524711,0.282338,0.701588
2019-11-05 10:12:00,-1.207146,1.231606,1.281895,0.627010,1.476913,-0.393653,-0.207550,0.955858


In [103]:
zeros

open         -0.032930
close         0.008369
hi           -0.645317
lo            0.627010
avg           0.002881
trades       -2.090240
avgVol       -5.857832
next_close    0.001017
dtype: float64

In [84]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,56656,-0.003532,0.994285,-0.491489,16.296609,-26.565229,10.551817
close,56656,0.001053,0.996984,-0.172631,47.422334,-33.142491,26.173658
hi,56656,0.001088,1.005060,4.882894,60.358916,-0.645317,34.691132
lo,56656,0.000652,0.987452,-7.031772,183.155173,-51.963985,0.627010
avg,56656,0.001016,0.994842,-1.116430,77.223916,-41.970133,20.224632
trades,56656,0.000881,1.001023,1.126718,3.159000,-2.090240,10.170139
avgVol,56656,0.000404,0.998795,-3.470566,18.580684,-5.857832,8.156366
next_close,56656,0.002124,0.996928,-0.394536,38.142905,-30.793261,25.710435


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [85]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.032930,-9.077997,-0.645317,-11.427499,-6.628619,1.661056,0.258170
2019-04-01 09:31:00,1.096683,2.569451,2.813446,-0.066756,0.893182,1.108876,0.301141
2019-04-01 09:32:00,1.771186,-1.179740,-0.275191,-2.606325,-1.585695,1.007291,-0.094274
2019-04-01 09:33:00,1.095026,-1.363172,-0.275036,-1.914133,-0.492748,0.848335,0.037132
2019-04-01 09:34:00,-1.613393,-0.540751,-0.645317,-1.222935,-1.369184,-0.131190,0.130155
...,...,...,...,...,...,...,...
2019-11-05 10:10:00,0.134665,0.008369,-0.645317,0.627010,0.002881,-1.524711,-4.396188
2019-11-05 10:11:00,-3.386070,0.008369,-0.645317,0.627010,0.002881,-1.524711,0.282338
2019-11-05 10:12:00,-1.207146,1.231606,1.281895,0.627010,1.476913,-0.393653,-0.207550


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [90]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(56656, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [163]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(45134, 30, 7), (45134,), (11522, 30, 7), (11522,), (56656, 30, 7), (56656,)]

In [109]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.60,191.189,4320,825935.940,44,191.645,190.98,1.001731
2019-04-01 09:31:00,190.700,190.98,190.980,190.64,190.761,3246,619210.510,32,190.650,190.93,0.999738
2019-04-01 09:32:00,191.060,190.93,191.090,190.78,190.951,2253,430211.740,30,190.980,190.83,0.999476
2019-04-01 09:33:00,190.980,190.83,191.010,190.76,190.946,2241,427911.290,27,190.930,190.70,0.999319
2019-04-01 09:34:00,190.760,190.70,190.760,190.60,190.666,1069,203822.465,12,190.830,190.85,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-11-05 10:11:00,256.840,256.84,256.840,256.84,256.840,100,25684.000,1,257.040,256.95,1.000428
2019-11-05 10:12:00,256.770,256.95,256.980,256.77,256.906,620,159282.060,9,256.840,257.10,1.000584
2019-11-05 10:13:00,257.020,257.10,257.100,257.01,257.044,704,180959.080,7,256.950,257.27,1.000661


## Build model

In [113]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [114]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(window, len(cols))))
model.add(Dense(1))
model.build()
model.compile(loss='mae', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [185]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.6445973863726133, 0.6388748551241561)

In [ ]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=100000,
          epochs=100)

px = model.predict(x)
res = np.swapaxes(array([x[:10, -1, 0], y[:10], px[:10, -1]]), 0, 1)
px.max() - px.min(), res

In [168]:
def diffs_df(model):
    def stats(x, y, name=None, predict=True):
        suffix = ' (%s)' % name if name else ''
        if predict:
            predx = model.predict(x)[:, 0]
        else:
            predx = x

        diffs = (y - predx)
        mae = abs(diffs)
        mse = mae**2

        return DF({
            ('y^' + suffix): predx,
            ('Δ' + suffix): diffs,
            ('|Δ|' + suffix): mae,
            ('Δ²' + suffix): mse,
        })

    train = stats(tx, ty, 't')
    val = stats(vx, vy, 'v')
    predx_t = train['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ])
    all = stats(predx, np.concatenate([ ty, vy ]), name='*', predict=False)

    return concat([
        desc(train),
        desc(val),
        desc(all)
    ])

In [169]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),45134,-0.145038,0.952837,0.192882,-1.217306,-1.782958e+00,1.770033
Δ (t),45134,0.145038,1.375148,-0.700510,11.120981,-3.017002e+01,24.299830
|Δ| (t),45134,1.082536,0.860325,3.591872,57.596879,1.552786e-05,30.170021
Δ² (t),45134,1.912026,6.783944,74.473274,8581.855764,2.411145e-10,910.230156
y^ (v),11522,-0.152145,0.951057,0.208306,-1.200379,-1.764462e+00,1.706371
Δ (v),11522,0.162590,1.363101,-0.638771,3.752013,-1.246926e+01,13.460422
|Δ| (v),11522,1.078965,0.848655,2.435612,15.780264,1.090562e-04,13.460422
Δ² (v),11522,1.884319,4.358953,16.199426,473.120968,1.189326e-08,181.182958
y^ (*),56656,-0.146483,0.952471,0.196016,-1.213928,-1.782958e+00,1.770033
Δ (*),56656,0.148607,1.372713,-0.688312,9.664875,-3.017002e+01,24.299830


In [170]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=100000,
          epochs=10)
diffs_df(model)

Train on 45134 samples, validate on 11522 samples
Epoch 1/10
45134/45134 [==============================] - 2s 49us/sample - loss: 1.0825 - val_loss: 1.0752
Epoch 2/10
45134/45134 [==============================] - 0s 9us/sample - loss: 1.0788 - val_loss: 1.0714
Epoch 3/10
45134/45134 [==============================] - 0s 7us/sample - loss: 1.0750 - val_loss: 1.0676
Epoch 4/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0713 - val_loss: 1.0639
Epoch 5/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0676 - val_loss: 1.0602
Epoch 6/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0639 - val_loss: 1.0565
Epoch 7/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0602 - val_loss: 1.0528
Epoch 8/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0566 - val_loss: 1.0491
Epoch 9/10
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0529 - val_loss: 1.

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),45134,-0.106508,0.910635,0.154301,-1.158887,-1.770296e+00,1.782657
Δ (t),45134,0.106508,1.346971,-0.670055,12.114660,-3.015135e+01,24.377137
|Δ| (t),45134,1.045745,0.855610,3.699520,59.486005,3.127027e-05,30.151347
Δ² (t),45134,1.825634,6.770984,75.050559,8651.792625,9.778297e-10,909.103721
y^ (v),11522,-0.112988,0.908780,0.169992,-1.140809,-1.756540e+00,1.702695
Δ (v),11522,0.123433,1.334558,-0.607489,4.070412,-1.245649e+01,13.430032
|Δ| (v),11522,1.041852,0.843049,2.519635,16.358631,1.701798e-04,13.430032
Δ² (v),11522,1.796126,4.318571,16.443103,484.617826,2.896118e-08,180.365759
y^ (*),56656,-0.107826,0.910254,0.157489,-1.155278,-1.770296e+00,1.782657
Δ (*),56656,0.109950,1.344461,-0.657713,10.527473,-3.015135e+01,24.377137


In [171]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=100000,
          epochs=100)
diffs_df(model)

Train on 45134 samples, validate on 11522 samples
Epoch 1/100
45134/45134 [==============================] - 0s 9us/sample - loss: 1.0457 - val_loss: 1.0383
Epoch 2/100
45134/45134 [==============================] - 0s 7us/sample - loss: 1.0422 - val_loss: 1.0347
Epoch 3/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0386 - val_loss: 1.0312
Epoch 4/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0351 - val_loss: 1.0277
Epoch 5/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0317 - val_loss: 1.0242
Epoch 6/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0282 - val_loss: 1.0208
Epoch 7/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0248 - val_loss: 1.0173
Epoch 8/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0213 - val_loss: 1.0139
Epoch 9/100
45134/45134 [==============================] - 0s 6us/sample - loss: 1.0180 - val_

Epoch 77/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8497 - val_loss: 0.8403
Epoch 78/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8479 - val_loss: 0.8385
Epoch 79/100
45134/45134 [==============================] - 0s 7us/sample - loss: 0.8462 - val_loss: 0.8368
Epoch 80/100
45134/45134 [==============================] - 0s 7us/sample - loss: 0.8445 - val_loss: 0.8351
Epoch 81/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8427 - val_loss: 0.8333
Epoch 82/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8411 - val_loss: 0.8316
Epoch 83/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8394 - val_loss: 0.8299
Epoch 84/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8377 - val_loss: 0.8283
Epoch 85/100
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8360 - val_loss: 0.8266
Epoch 86/100
45134/45134 [==

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),45134,0.058952,0.557213,-0.000992,-0.266895,-1.619649e+00,1.766364
Δ (t),45134,-0.058952,1.143628,-0.428578,24.272285,-3.016512e+01,25.249830
|Δ| (t),45134,0.811352,0.808118,4.665530,81.003225,1.727643e-05,30.165124
Δ² (t),45134,1.311332,6.716306,80.678163,9426.946355,2.984751e-10,909.934709
y^ (v),11522,0.057814,0.554668,-0.007557,-0.202472,-1.592843e+00,1.570629
Δ (v),11522,-0.047370,1.128063,-0.352184,8.079378,-1.249395e+01,13.401036
|Δ| (v),11522,0.803478,0.793182,3.179396,22.518760,2.179861e-04,13.401036
Δ² (v),11522,1.274660,4.053094,19.212934,627.061650,4.751793e-08,179.587755
y^ (*),56656,0.058720,0.556691,-0.002304,-0.253939,-1.619649e+00,1.766364
Δ (*),56656,-0.056596,1.140480,-0.413688,21.123758,-3.016512e+01,25.249830


In [177]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=100000,
          epochs=1000)
diffs_df(model)

Train on 45134 samples, validate on 11522 samples
Epoch 1/1000
45134/45134 [==============================] - 1s 13us/sample - loss: 0.8114 - val_loss: 0.8021
Epoch 2/1000
45134/45134 [==============================] - 0s 11us/sample - loss: 0.8099 - val_loss: 0.8006
Epoch 3/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8085 - val_loss: 0.7992
Epoch 4/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8071 - val_loss: 0.7979
Epoch 5/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.8057 - val_loss: 0.7965
Epoch 6/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.8043 - val_loss: 0.7951
Epoch 7/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.8030 - val_loss: 0.7938
Epoch 8/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.8016 - val_loss: 0.7925
Epoch 9/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.

Epoch 76/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7347 - val_loss: 0.7265
Epoch 77/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7341 - val_loss: 0.7258
Epoch 78/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7334 - val_loss: 0.7251
Epoch 79/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7327 - val_loss: 0.7245
Epoch 80/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7320 - val_loss: 0.7238
Epoch 81/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7314 - val_loss: 0.7231
Epoch 82/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.7307 - val_loss: 0.7225
Epoch 83/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.7301 - val_loss: 0.7218
Epoch 84/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.7294 - val_loss: 0.7212
Epoch 85/1000
45134

Epoch 151/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6980 - val_loss: 0.6904
Epoch 152/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6977 - val_loss: 0.6900
Epoch 153/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6973 - val_loss: 0.6897
Epoch 154/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6970 - val_loss: 0.6894
Epoch 155/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6967 - val_loss: 0.6891
Epoch 156/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6964 - val_loss: 0.6888
Epoch 157/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6960 - val_loss: 0.6885
Epoch 158/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6957 - val_loss: 0.6882
Epoch 159/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6954 - val_loss: 0.6879
Epoch 160/

Epoch 226/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6798 - val_loss: 0.6729
Epoch 227/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6796 - val_loss: 0.6727
Epoch 228/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6795 - val_loss: 0.6726
Epoch 229/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6793 - val_loss: 0.6724
Epoch 230/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6791 - val_loss: 0.6722
Epoch 231/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6789 - val_loss: 0.6721
Epoch 232/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6788 - val_loss: 0.6719
Epoch 233/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6786 - val_loss: 0.6718
Epoch 234/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6785 - val_loss: 0.6716
Epoch 235/

Epoch 301/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6697 - val_loss: 0.6630
Epoch 302/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6696 - val_loss: 0.6629
Epoch 303/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6695 - val_loss: 0.6628
Epoch 304/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6693 - val_loss: 0.6627
Epoch 305/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6692 - val_loss: 0.6626
Epoch 306/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6691 - val_loss: 0.6625
Epoch 307/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6690 - val_loss: 0.6624
Epoch 308/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6689 - val_loss: 0.6623
Epoch 309/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6688 - val_loss: 0.6622
Epoch 310/

...

Epoch 751/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 752/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 753/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 754/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 755/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 756/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 757/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6431
Epoch 758/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6494 - val_loss: 0.6430
Epoch 759/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6493 - val_loss: 0.6430
Epoch 760/

Epoch 826/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6486 - val_loss: 0.6422
Epoch 827/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6486 - val_loss: 0.6422
Epoch 828/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 829/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 830/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 831/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 832/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 833/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 834/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6485 - val_loss: 0.6422
Epoch 835/

Epoch 901/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6479 - val_loss: 0.6415
Epoch 902/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6479 - val_loss: 0.6415
Epoch 903/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6415
Epoch 904/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6479 - val_loss: 0.6415
Epoch 905/1000
45134/45134 [==============================] - 0s 8us/sample - loss: 0.6478 - val_loss: 0.6414
Epoch 906/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6478 - val_loss: 0.6414
Epoch 907/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6478 - val_loss: 0.6414
Epoch 908/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6478 - val_loss: 0.6414
Epoch 909/1000
45134/45134 [==============================] - 0s 8us/sample - loss: 0.6478 - val_loss: 0.6414
Epoch 910/

Epoch 976/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6409
Epoch 977/1000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6473 - val_loss: 0.6409
Epoch 978/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 979/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 980/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 981/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 982/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 983/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 984/1000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6473 - val_loss: 0.6408
Epoch 985/

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),45134,0.011477,0.077280,0.080204,5.261648,-3.793658e-01,1.048494
Δ (t),45134,-0.011477,1.001358,-0.438369,44.252325,-3.090069e+01,25.978794
|Δ| (t),45134,0.647141,0.764232,5.727502,112.122482,4.565120e-06,30.900692
Δ² (t),45134,1.002828,6.820838,88.452351,10791.234173,2.084032e-11,954.852747
y^ (v),11522,0.011654,0.079140,0.128578,5.027551,-5.019673e-01,0.786491
Δ (v),11522,-0.001209,0.986503,-0.279681,14.849632,-1.251443e+01,13.693045
|Δ| (v),11522,0.640692,0.750112,3.916245,32.110943,1.166871e-04,13.693045
Δ² (v),11522,0.973105,3.994760,22.187477,770.131353,1.361587e-08,187.499475
y^ (*),56656,0.011513,0.077661,0.090677,5.213497,-5.019673e-01,1.048494
Δ (*),56656,-0.009389,0.998355,-0.407397,38.558871,-3.090069e+01,25.978794


In [182]:
model_ckpt_path = data_dir / now.strftime('%Y-%m-%dT%H:%M.ckpt')

In [184]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=100000,
          epochs=10000,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_best_only=True, period=20),
          ]
         )
diffs_df(model)

Train on 45134 samples, validate on 11522 samples
Epoch 1/10000
45134/45134 [==============================] - 1s 18us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 2/10000
45134/45134 [==============================] - 0s 8us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 3/10000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 4/10000
45134/45134 [==============================] - 1s 20us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 5/10000
45134/45134 [==============================] - 1s 17us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 6/10000
45134/45134 [==============================] - 1s 12us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 7/10000
45134/45134 [==============================] - 0s 8us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 8/10000
45134/45134 [==============================] - 0s 8us/sample - loss: 0.6469 - val_loss: 0.6404
Epoch 9/10000
45134/45134 [==============================] - 0s 8us/sample

45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 72/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 73/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 74/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 75/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 76/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 77/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 78/10000
45134/45134 [==============================] - 0s 5us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 79/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6465 - val_loss: 0.6400
Epoch 80/10000
INFO:tenso

Epoch 142/10000
45134/45134 [==============================] - 0s 10us/sample - loss: 0.6462 - val_loss: 0.6397
Epoch 143/10000
45134/45134 [==============================] - 0s 9us/sample - loss: 0.6462 - val_loss: 0.6397
Epoch 144/10000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6462 - val_loss: 0.6397
Epoch 145/10000
45134/45134 [==============================] - 1s 17us/sample - loss: 0.6462 - val_loss: 0.6397
Epoch 146/10000
45134/45134 [==============================] - 1s 12us/sample - loss: 0.6462 - val_loss: 0.6397
Epoch 147/10000
45134/45134 [==============================] - 0s 9us/sample - loss: 0.6462 - val_loss: 0.6396
Epoch 148/10000
45134/45134 [==============================] - 1s 22us/sample - loss: 0.6462 - val_loss: 0.6396
Epoch 149/10000
45134/45134 [==============================] - 1s 11us/sample - loss: 0.6462 - val_loss: 0.6396
Epoch 150/10000
45134/45134 [==============================] - 0s 11us/sample - loss: 0.6462 - val_loss: 0.

Epoch 213/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 214/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 215/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 216/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 217/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 218/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6394
Epoch 219/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6393
Epoch 220/10000
INFO:tensorflow:Assets written to: /Users/ryan/c/pipelines/notebooks/data/2019-11-05T15:16.ckpt/assets
45134/45134 [==============================] - 5s 100us/sample - loss: 0.6459 - val_loss: 0.6393
Epoch 2

45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 284/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 285/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 286/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 287/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 288/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 289/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 290/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 291/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6457 - val_loss: 0.6391
Epoch 292/10000
4

...

Epoch 1911/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1912/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1913/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1914/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1915/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1916/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1917/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1918/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1919/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss:

Epoch 1981/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1982/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1983/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1984/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1985/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1986/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1987/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1988/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 1989/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss:

Epoch 2053/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2054/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2055/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2056/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2057/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2058/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2059/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2060/10000
INFO:tensorflow:Assets written to: /Users/ryan/c/pipelines/notebooks/data/2019-11-05T15:16.ckpt/assets
45134/45134 [==============================] - 2s 35us/sample - loss: 0.6443 - val_loss: 0.6380


Epoch 2126/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2127/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2128/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2129/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2130/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2131/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2132/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2133/10000
45134/45134 [==============================] - 0s 7us/sample - loss: 0.6443 - val_loss: 0.6380
Epoch 2134/10000
45134/45134 [==============================] - 0s 6us/sample - loss: 0.6443 - val_loss:

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),45134,0.015070,0.032358,0.877290,3.947733,-1.286414e-01,0.501033
Δ (t),45134,-0.015070,0.999454,-0.458308,44.199871,-3.089476e+01,25.803216
|Δ| (t),45134,0.644319,0.764185,5.703543,111.179175,1.762261e-06,30.894756
Δ² (t),45134,0.999113,6.791646,88.572614,10847.321309,3.105564e-12,954.485961
y^ (v),11522,0.014867,0.033076,0.921381,3.550753,-1.126055e-01,0.309953
Δ (v),11522,-0.004422,0.984253,-0.306613,14.909026,-1.261044e+01,13.486893
|Δ| (v),11522,0.638019,0.749447,3.919305,32.072201,5.759100e-06,13.486893
Δ² (v),11522,0.968690,3.984061,21.928893,746.226650,3.316723e-11,181.896295
y^ (*),56656,0.015029,0.032505,0.886697,3.862932,-1.286414e-01,0.501033
Δ (*),56656,-0.012905,0.996382,-0.428745,38.535230,-3.089476e+01,25.803216


In [188]:
hist = DF(model.history.history)
hist.iloc[-110:-90]

,loss,val_loss
2049,0.644320,0.638020
2050,0.644320,0.638019
2051,0.644320,0.638020
2052,0.644319,0.638020
2053,0.644319,0.638019
2054,0.644319,0.638019
2055,0.644319,0.638019
2056,0.644319,0.638019
2057,0.644319,0.638019
2058,0.644319,0.638019


In [189]:
hist.iloc[-200:]

,loss,val_loss
1959,0.644332,0.638023
1960,0.644332,0.638023
1961,0.644331,0.638023
1962,0.644331,0.638023
1963,0.644331,0.638023
...,...,...
2154,0.644307,0.638026
2155,0.644307,0.638026
2156,0.644307,0.638026
2157,0.644307,0.638026


In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)